In [1]:
import scipy
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

In [2]:
# Create a 3D input
def create_dataset (X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

# Function for LSTM and GRU model
def create_model(units, m):
    model = Sequential()
    model.add(m (units = units, return_sequences = True,
                input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.2))
    initializer = tf.keras.initializers.GlorotUniform()
    model.add(m (units = units, kernel_initializer=initializer))
    #model.add(m (units = units))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1, activation = "sigmoid"))
    
    #opt = keras.optimizers.Adam(learning_rate=0.01)
    #Compile model
    model.compile(loss='mae', optimizer="adam")
    return model

# Fitting LSTM and GRU
def fit_model(model):
    earlystopping  = keras.callbacks.EarlyStopping(monitor = 'val_loss',
   
   #The patience for early stopping is set to 20 as the validation and training loss tends to zig zag.
                                               patience = 20)
    
    #Epochs at 150 is not too high considering the batch_size, steps_per_epoch and timestep set earlier.
    #Early stopping is meant to prevent overfitting.
    history = model.fit(X_train, y_train, epochs = 150,  
                        validation_split = 0.2, batch_size = 64, 
                        shuffle = True, callbacks = [earlystopping],
                        steps_per_epoch = 16, verbose = 0)
    return history
# Plot train loss and validation loss
def plot_loss (history):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
    
# Make prediction
def prediction(model):
    prediction = model.predict(X_test)
    prediction = scaler_y.inverse_transform(prediction)
    return prediction


# Plot true future vs prediction
def plot_future(prediction, y_test):
    plt.figure(figsize=(10, 6))
    range_future = len(prediction)
        
    plt.plot(np.arange(range_future), np.array(y_test), 
             label='True Future')
    
    plt.plot(np.arange(range_future),np.array(prediction),
            label='Prediction')
    
    plt.legend(loc='upper left')
    plt.xlabel('Time (hour)')
    plt.ylabel('Bike users per time period') 
    
# Define a function to calculate MAE and RMSE
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))
    print('')    

In [3]:
timesteps = [1, 6, 12, 18, 24]
column_names = ["GRU RMSE", "GRU MAE","GRU R-Squared", "LSTM RMSE", "LSTM MAE", "LSTM R-Squared"]
metrics = pd.DataFrame(columns = column_names)

for g in timesteps:
    GRU_RMSE = []
    GRU_MAE = []
    GRU_rsquared = []
    
    LSTM_RMSE = []
    LSTM_MAE = []
    LSTM_rsquared = []

    repeats = 5
    for i in range (repeats):
        df = pd.read_csv("08.06.22.csv", index_col = 0)

        train_size = int(len(df)*0.8)
        train_dataset, test_dataset = df.iloc[:train_size], df.iloc[train_size:]

        # Split train data to X and y
        X_train = train_dataset.drop(['Users'], axis = 1)
        y_train = train_dataset.loc[:,['Users']]

        # Split test data to X and y
        X_test = test_dataset.drop(['Users'], axis = 1)
        y_test = test_dataset.loc[:,['Users']]

        # Different scaler for input and output
        scaler_x = MinMaxScaler(feature_range = (0,1))
        scaler_y = MinMaxScaler(feature_range = (0,1))

        # Fit the scaler using available training data
        input_scaler = scaler_x.fit(X_train)
        output_scaler = scaler_y.fit(y_train)

        # Apply the scaler to training data
        train_y_norm = output_scaler.transform(y_train)
        train_x_norm = input_scaler.transform(X_train)

        # Apply the scaler to test data
        test_y_norm = output_scaler.transform(y_test)
        test_x_norm = input_scaler.transform(X_test)

        #Have set time steps to 12, equivalent of two days
        TIME_STEPS = g
        X_test, y_test = create_dataset(test_x_norm, test_y_norm,   
                                        TIME_STEPS)
        X_train, y_train = create_dataset(train_x_norm, train_y_norm, 
                                          TIME_STEPS)

        # GRU and LSTM
        model_gru = create_model(64, GRU)
        model_lstm = create_model(64, LSTM)

        history_lstm = fit_model(model_lstm)
        history_gru = fit_model(model_gru)


        y_test = scaler_y.inverse_transform(y_test)
        y_train = scaler_y.inverse_transform(y_train)

        prediction_lstm = prediction(model_lstm)
        prediction_gru = prediction(model_gru)

        errors_gru = prediction_gru - y_test
        errors_lstm = prediction_lstm - y_test
        
        
    
        GRU_RMSE.append(np.sqrt(np.square(errors_gru).mean()))
        GRU_MAE.append(np.abs(errors_gru).mean())
        GRU_rsquared.append(r2_score(y_test, prediction_gru))
        LSTM_RMSE.append(np.sqrt(np.square(errors_lstm).mean()))
        LSTM_MAE.append(np.abs(errors_lstm).mean())
        LSTM_rsquared.append(r2_score(y_test, prediction_lstm))
        
    GRU_RMSE_values = np.mean(GRU_RMSE)
    GRU_MAE_values = np.mean(GRU_MAE)
    GRU_rsquared_values = np.mean(GRU_rsquared)
    
    LSTM_RMSE_values = np.mean(LSTM_RMSE)
    LSTM_MAE_values = np.mean(LSTM_MAE)
    LSTM_rsquared_values = np.mean(LSTM_rsquared)

    metrics.loc[len(metrics)] = [GRU_RMSE_values, GRU_MAE_values,GRU_rsquared_values, 
                                 LSTM_RMSE_values, LSTM_MAE_values, LSTM_rsquared_values]
metrics

,GRU RMSE,GRU MAE,GRU R-Squared,LSTM RMSE,LSTM MAE,LSTM R-Squared
0,10.879885,7.047080,0.727464,11.352778,7.282853,0.703094
1,12.178848,7.909175,0.651431,14.675442,8.759102,0.499166
2,12.001689,7.713824,0.664303,13.916958,8.491255,0.539669
3,12.147977,7.706137,0.651146,11.517268,7.347827,0.687726
4,11.884082,7.722102,0.658138,11.976467,7.841743,0.660832


In [4]:
metrics.insert(loc = 0, column = "Time steps", value = timesteps)

In [5]:
metrics

,Time steps,GRU RMSE,GRU MAE,GRU R-Squared,LSTM RMSE,LSTM MAE,LSTM R-Squared
0,1,10.879885,7.047080,0.727464,11.352778,7.282853,0.703094
1,6,12.178848,7.909175,0.651431,14.675442,8.759102,0.499166
2,12,12.001689,7.713824,0.664303,13.916958,8.491255,0.539669
3,18,12.147977,7.706137,0.651146,11.517268,7.347827,0.687726
4,24,11.884082,7.722102,0.658138,11.976467,7.841743,0.660832
